In [1]:

import sys
import os
notebookDir = os.getcwd()
parentDir = os.path.dirname(notebookDir)
sys.path.append(parentDir)

from src.data import fetch_data, clean_data, normalize_data, robust_normalize_data
from src.features import add_binary_targets, add_lag_delta, calculate_binary_variables, get_close_direction

# Now you can use the functions
rawPath = '../data/ES/CME_MINI_ESX_5m.csv'

rawES = fetch_data(rawPath)
df = clean_data(rawES)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1648349 entries, 0 to 1648348
Data columns (total 13 columns):
 #   Column      Non-Null Count    Dtype              
---  ------      --------------    -----              
 0   DateTime    1648349 non-null  datetime64[ns, UTC]
 1   Open        1648349 non-null  float64            
 2   High        1648349 non-null  float64            
 3   Low         1648349 non-null  float64            
 4   Close       1648349 non-null  float64            
 5   HourOfDay   1648349 non-null  int64              
 6   Minute      1648349 non-null  int64              
 7   DayOfWeek   1648349 non-null  int64              
 8   Day         1648349 non-null  int64              
 9   Month       1648349 non-null  int64              
 10  Year        1648349 non-null  int64              
 11  WeekOfYear  1648349 non-null  UInt32             
 12  xClose      1648349 non-null  int64              
dtypes: UInt32(1), datetime64[ns, UTC](1), float64(4), int64(7

In [2]:
df = calculate_binary_variables(df)

/Users/zSpinach/Documents/Dev/zSPH/QuantumBTP/src/features.py:148: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['DH'] = df.groupby(df['DateTime'].dt.to_period("D"))['High'].transform('max')
/Users/zSpinach/Documents/Dev/zSPH/QuantumBTP/src/features.py:149: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['DL'] = df.groupby(df['DateTime'].dt.to_period("D"))['Low'].transform('min')


In [3]:
print(df.value_counts(df['DH'] == 1))
print(df.value_counts(df['DL'] == 1))
df

DH
False    1635749
True       12600
Name: count, dtype: int64
DL
False    1636604
True       11745
Name: count, dtype: int64


,DateTime,Open,High,Low,Close,HourOfDay,Minute,DayOfWeek,Day,Month,Year,WeekOfYear,xClose,DH,DL
0,2000-01-03 06:15:00+00:00,1487.25,1487.75,1487.00,1487.75,6,15,0,3,1,2000,1,1,0,0
1,2000-01-03 06:20:00+00:00,1488.25,1488.50,1488.25,1488.50,6,20,0,3,1,2000,1,1,0,0
2,2000-01-03 06:25:00+00:00,1488.75,1488.75,1488.50,1488.50,6,25,0,3,1,2000,1,0,0,0
3,2000-01-03 06:30:00+00:00,1488.25,1488.50,1488.25,1488.50,6,30,0,3,1,2000,1,1,0,0
4,2000-01-03 06:35:00+00:00,1488.00,1488.50,1488.00,1488.50,6,35,0,3,1,2000,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648344,2024-03-08 16:35:00+00:00,5133.75,5134.00,5133.00,5133.00,16,35,4,8,3,2024,10,0,0,0
1648345,2024-03-08 16:40:00+00:00,5133.00,5133.50,5132.75,5133.50,16,40,4,8,3,2024,10,1,0,0
1648346,2024-03-08 16:45:00+00:00,5133.25,5134.25,5133.00,5133.75,16,45,4,8,3,2024,10,1,0,0
1648347,2024-03-08 16:50:00+00:00,5134.00,5134.00,5133.25,5133.50,16,50,4,8,3,2024,10,0,0,0


In [4]:
train_df = df.set_index('DateTime')
print(df.info())
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1648349 entries, 0 to 1648348
Data columns (total 15 columns):
 #   Column      Non-Null Count    Dtype              
---  ------      --------------    -----              
 0   DateTime    1648349 non-null  datetime64[ns, UTC]
 1   Open        1648349 non-null  float64            
 2   High        1648349 non-null  float64            
 3   Low         1648349 non-null  float64            
 4   Close       1648349 non-null  float64            
 5   HourOfDay   1648349 non-null  int64              
 6   Minute      1648349 non-null  int64              
 7   DayOfWeek   1648349 non-null  int64              
 8   Day         1648349 non-null  int64              
 9   Month       1648349 non-null  int64              
 10  Year        1648349 non-null  int64              
 11  WeekOfYear  1648349 non-null  UInt32             
 12  xClose      1648349 non-null  int64              
 13  DH          1648349 non-null  int64              
 14  DL

In [5]:
df

,DateTime,Open,High,Low,Close,HourOfDay,Minute,DayOfWeek,Day,Month,Year,WeekOfYear,xClose,DH,DL
0,2000-01-03 06:15:00+00:00,1487.25,1487.75,1487.00,1487.75,6,15,0,3,1,2000,1,1,0,0
1,2000-01-03 06:20:00+00:00,1488.25,1488.50,1488.25,1488.50,6,20,0,3,1,2000,1,1,0,0
2,2000-01-03 06:25:00+00:00,1488.75,1488.75,1488.50,1488.50,6,25,0,3,1,2000,1,0,0,0
3,2000-01-03 06:30:00+00:00,1488.25,1488.50,1488.25,1488.50,6,30,0,3,1,2000,1,1,0,0
4,2000-01-03 06:35:00+00:00,1488.00,1488.50,1488.00,1488.50,6,35,0,3,1,2000,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648344,2024-03-08 16:35:00+00:00,5133.75,5134.00,5133.00,5133.00,16,35,4,8,3,2024,10,0,0,0
1648345,2024-03-08 16:40:00+00:00,5133.00,5133.50,5132.75,5133.50,16,40,4,8,3,2024,10,1,0,0
1648346,2024-03-08 16:45:00+00:00,5133.25,5134.25,5133.00,5133.75,16,45,4,8,3,2024,10,1,0,0
1648347,2024-03-08 16:50:00+00:00,5134.00,5134.00,5133.25,5133.50,16,50,4,8,3,2024,10,0,0,0


In [6]:
train_df

,Open,High,Low,Close,HourOfDay,Minute,DayOfWeek,Day,Month,Year,WeekOfYear,xClose,DH,DL
DateTime,,,,,,,,,,,,,,
2000-01-03 06:15:00+00:00,1487.25,1487.75,1487.00,1487.75,6,15,0,3,1,2000,1,1,0,0
2000-01-03 06:20:00+00:00,1488.25,1488.50,1488.25,1488.50,6,20,0,3,1,2000,1,1,0,0
2000-01-03 06:25:00+00:00,1488.75,1488.75,1488.50,1488.50,6,25,0,3,1,2000,1,0,0,0
2000-01-03 06:30:00+00:00,1488.25,1488.50,1488.25,1488.50,6,30,0,3,1,2000,1,1,0,0
2000-01-03 06:35:00+00:00,1488.00,1488.50,1488.00,1488.50,6,35,0,3,1,2000,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-08 16:35:00+00:00,5133.75,5134.00,5133.00,5133.00,16,35,4,8,3,2024,10,0,0,0
2024-03-08 16:40:00+00:00,5133.00,5133.50,5132.75,5133.50,16,40,4,8,3,2024,10,1,0,0
2024-03-08 16:45:00+00:00,5133.25,5134.25,5133.00,5133.75,16,45,4,8,3,2024,10,1,0,0


In [7]:
df = get_close_direction(df)

/Users/zSpinach/Documents/Dev/zSPH/QuantumBTP/src/features.py:168: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  daily_stats = df.groupby(df['DateTime'].dt.to_period("D")).agg({'Open': 'first', 'Close': 'last'})
/Users/zSpinach/Documents/Dev/zSPH/QuantumBTP/src/features.py:172: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['DR'] = df['DateTime'].dt.to_period("D").map(daily_return)


In [8]:
df

,DateTime,Open,High,Low,Close,HourOfDay,Minute,DayOfWeek,Day,Month,Year,WeekOfYear,xClose,DH,DL,DR
0,2000-01-03 06:15:00+00:00,1487.25,1487.75,1487.00,1487.75,6,15,0,3,1,2000,1,1,0,0,0
1,2000-01-03 06:20:00+00:00,1488.25,1488.50,1488.25,1488.50,6,20,0,3,1,2000,1,1,0,0,0
2,2000-01-03 06:25:00+00:00,1488.75,1488.75,1488.50,1488.50,6,25,0,3,1,2000,1,0,0,0,0
3,2000-01-03 06:30:00+00:00,1488.25,1488.50,1488.25,1488.50,6,30,0,3,1,2000,1,1,0,0,0
4,2000-01-03 06:35:00+00:00,1488.00,1488.50,1488.00,1488.50,6,35,0,3,1,2000,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648344,2024-03-08 16:35:00+00:00,5133.75,5134.00,5133.00,5133.00,16,35,4,8,3,2024,10,0,0,0,0
1648345,2024-03-08 16:40:00+00:00,5133.00,5133.50,5132.75,5133.50,16,40,4,8,3,2024,10,1,0,0,0
1648346,2024-03-08 16:45:00+00:00,5133.25,5134.25,5133.00,5133.75,16,45,4,8,3,2024,10,1,0,0,0
1648347,2024-03-08 16:50:00+00:00,5134.00,5134.00,5133.25,5133.50,16,50,4,8,3,2024,10,0,0,0,0


In [9]:
print(df.value_counts(df['DR'] == 1))

DR
True     883622
False    764727
Name: count, dtype: int64


In [10]:
# Step 2: Group by day and retrieve the DR value for each day
daily_dr = df.groupby(df['DateTime'].dt.to_period('D'))['DR'].first()

# Step 3: Count the number of up days and down days
up_days_count = daily_dr.sum()
down_days_count = (daily_dr == 0).sum()

# Display the results
print(f"Number of Up Days: {up_days_count}")
print(f"Number of Down Days: {down_days_count}")

Number of Up Days: 4000
Number of Down Days: 3503


/var/folders/x_/hkzshp5n7nj5_k4zdq_p25ww0000gn/T/ipykernel_6905/1273888525.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  daily_dr = df.groupby(df['DateTime'].dt.to_period('D'))['DR'].first()


In [11]:
train_df["DR"] = df["DR"].copy().astype(int)

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1648349 entries, 2000-01-03 06:15:00+00:00 to 2024-03-08 16:55:00+00:00
Data columns (total 15 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Open        1648349 non-null  float64
 1   High        1648349 non-null  float64
 2   Low         1648349 non-null  float64
 3   Close       1648349 non-null  float64
 4   HourOfDay   1648349 non-null  int64  
 5   Minute      1648349 non-null  int64  
 6   DayOfWeek   1648349 non-null  int64  
 7   Day         1648349 non-null  int64  
 8   Month       1648349 non-null  int64  
 9   Year        1648349 non-null  int64  
 10  WeekOfYear  1648349 non-null  UInt32 
 11  xClose      1648349 non-null  int64  
 12  DH          1648349 non-null  int64  
 13  DL          1648349 non-null  int64  
 14  DR          0 non-null        float64
dtypes: UInt32(1), float64(5), int64(9)
memory usage: 196.5 MB


In [13]:
robust_normalize_data(train_df)
train_df

,Open,High,Low,Close,HourOfDay,Minute,DayOfWeek,Day,Month,Year,WeekOfYear,xClose,DH,DL,DR
DateTime,,,,,,,,,,,,,,,
2000-01-03 06:15:00+00:00,-0.001185,-0.001185,-0.001017,-0.000847,6,15,0,3,1,2000,1,1,0,0,NaN
2000-01-03 06:20:00+00:00,-0.000508,-0.000677,-0.000169,-0.000339,6,20,0,3,1,2000,1,1,0,0,NaN
2000-01-03 06:25:00+00:00,-0.000169,-0.000508,0.000000,-0.000339,6,25,0,3,1,2000,1,0,0,0,NaN
2000-01-03 06:30:00+00:00,-0.000508,-0.000677,-0.000169,-0.000339,6,30,0,3,1,2000,1,1,0,0,NaN
2000-01-03 06:35:00+00:00,-0.000677,-0.000677,-0.000339,-0.000339,6,35,0,3,1,2000,1,1,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-08 16:35:00+00:00,2.468925,2.467084,2.470010,2.468417,16,35,4,8,3,2024,10,0,0,0,NaN
2024-03-08 16:40:00+00:00,2.468417,2.466746,2.469841,2.468755,16,40,4,8,3,2024,10,1,0,0,NaN
2024-03-08 16:45:00+00:00,2.468586,2.467253,2.470010,2.468925,16,45,4,8,3,2024,10,1,0,0,NaN


In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1648349 entries, 2000-01-03 06:15:00+00:00 to 2024-03-08 16:55:00+00:00
Data columns (total 15 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Open        1648349 non-null  float64
 1   High        1648349 non-null  float64
 2   Low         1648349 non-null  float64
 3   Close       1648349 non-null  float64
 4   HourOfDay   1648349 non-null  int64  
 5   Minute      1648349 non-null  int64  
 6   DayOfWeek   1648349 non-null  int64  
 7   Day         1648349 non-null  int64  
 8   Month       1648349 non-null  int64  
 9   Year        1648349 non-null  int64  
 10  WeekOfYear  1648349 non-null  UInt32 
 11  xClose      1648349 non-null  int64  
 12  DH          1648349 non-null  int64  
 13  DL          1648349 non-null  int64  
 14  DR          0 non-null        float64
dtypes: UInt32(1), float64(5), int64(9)
memory usage: 196.5 MB


In [16]:
train_df["DR"] = get_close_direction(train_df)
train_df

KeyError: 'DateTime'